# PE Ratio Analysis
Calculate historical PE ratio for stocks 

[FAST Graphs: Earnings and Price Correlation](https://www.fastgraphs.com/wp/why-doesnt-the-normal-pe-in-the-historical-graph-match-the-normal-pe-in-the-forecasting-graph/)

In [ ]:
library(xts)
library(zoo)
library(TTR)
library(XML)
library(rlist)
library(dplyr)
library(quantmod)
library(reshape2)
library(XLConnect)
library(data.table)
setwd('/Users/jtan/Documents/Investment/Price-Earnings-Ratio')

# avoid scientific number format
options(scipen=999)

In [ ]:
STOCK_LIST = c('MSFT')
START_DATE <- '2016-01-01'
END_DATE <- '2017-01-01'

output_table <- data.frame (
    symbol=character(),
    period=character(),
    metric=character(),
    value=numeric(),
    stringsAsFactors=FALSE
)

In [ ]:
# period: Y = annual reporting, Q = quarterly reporting
# report_type: Income Statement, Cash Flow, Balance Sheet
get_stockrow_data <- function(symbol, report_type, report) {
    # http://stockrow.com/api/companies/NVDA/financials.xlsx?dimension=MRQ&section=Income Statement
    url <- paste('http://stockrow.com/api/companies/', symbol, '/financials.xlsx?dimension=MR', report, '&section=', report_type, sep = '')
    download.file(url, 'temp.xlsx', mode="wb")
    readWorksheet(loadWorkbook('temp.xlsx'), sheet = symbol, header = TRUE)
}

reshape_stockrow_data <- function(data, symbol, report, metric_array) {
    # selecting the desired metric
    colnames(data)[1] <- 'Metric'
    data <- filter(data, data$Metric %in% metric_array) 
    # fix column names
    name_array <- colnames(data)
    for (i in 2:length(name_array)) { 
        name_array[i] <- substr(colnames(data)[i], 2, 8) 
    }
    name_array <- gsub("\\.", "\\-", name_array)
    colnames(data) <- name_array
    # reshape and organize data
    data <- melt(data, id=c("Metric"))
    colnames(data)[2] <- 'Period'
    data$Symbol <- symbol
    data$value <- ifelse(abs(data$value) > 1000, as.integer(data$value/1000000), data$value)
    data <- data[c('Symbol','Period','Metric','value')]
}

get_earnings_date <- function(symbol) {
    url <- paste('http://www.nasdaq.com/symbol/', symbol, '/earnings-surprise', sep='')
    tables <- readHTMLTable(url)
    tables <- list.clean(tables, fun = is.null, recursive = FALSE)
    n.cols <- unlist(lapply(tables, function(t) dim(t)[2]))
    data <- tables[[which.max(n.cols)]]
}

reshape_earnings_date <- function(data) {
    # keep only what is needed
    data <- data[-1,1:2]
    colnames(data)[1] <- 'Period'
    colnames(data)[2] <- 'DateReported'
    data <- data.frame(lapply(data, as.character), stringsAsFactors=FALSE)
    data$DateReported <- as.Date(data$DateReported, '%m/%d/%Y')
    data.month <- substr(data$Period,1,3) 
    data.month <- match(data.month,month.abb) 
    data.month <- sprintf("%02d", data.month)
    data.year <- substr(data$Period,4,7)
    data$Period <- paste(data.year, data.month, sep='-')
    data
}


In [ ]:
for(i in 1:length(STOCK_LIST)) {

    cat('Processing', STOCK_LIST[i],'\n')

    # get pricing data
    data.price <- getSymbols(STOCK_LIST[i], from=START_DATE, to=END_DATE, auto.assign=FALSE)
    data.price <- data.frame(Date = index(data.price), data.price, row.names=NULL)
    colnames(data.price) <- c('Date','Open','High','Low','Close','Volume','Adjusted')
    data.price <- data.price[order(data.price$Date, decreasing=TRUE),]
    data.price$Symbol <- STOCK_LIST[i]
    
    # get earnings data
    stockrow_table <- output_table # needed when reading from multiple report_types
    data.is <- get_stockrow_data(STOCK_LIST[i], 'Income Statement', 'Q')
    data.is <- reshape_stockrow_data(data.is, STOCK_LIST[i], 'Q', c('Revenues','EPS','EBITDA'))
    stockrow_table <- rbind(data.is, stockrow_table)
    data.date <- get_earnings_date(STOCK_LIST[i])
    data.date <- reshape_earnings_date (data.date)
    
    unique <- stockrow_table %>% distinct(stockrow_table$Period) %>% mutate_if(is.factor, as.character)
    colnames(unique)[1] <- 'Period'
    new <- merge(unique, data.date, by='Period', all=TRUE)
    new <- new[order(new$Period, decreasing=TRUE),]
    for(i in min(which(is.na(new$DateReported))):nrow(new)) {
        new$DateReported[i] <- as.yearmon(new$DateReported[i-4]) - 1 
        new$DateReported[i] <- as.Date(new$DateReported[i]) + as.numeric(format(new$DateReported[i-4],'%d')) - 1
    }    
    data.eps.quarter <- merge(stockrow_table, new, by=c('Period'), all=TRUE)
    
    # start by only using EPS
    data.eps.quarter <- data.eps.quarter %>% filter(Metric == 'EPS') %>% select(DateReported, value) %>% arrange(desc(DateReported))  
    data.eps.annual <- data.frame(matrix(NA, nrow=nrow(data.eps.quarter)-3, ncol=2)) 
    colnames(data.eps.annual) <- c('Date','EPS')
    data.eps.annual$Date <- head(data.eps.quarter$DateReported, -3)
    data.eps.annual$EPS <- rollapply(data.eps.quarter$value, width = 4, by = 1, FUN = sum, align = "left")
    data.eps.annual$Symbol <- STOCK_LIST[i]
    
    # merge/combine data
    data.combine <- merge(data.price, data.eps.annual, by=c('Symbol','Date'), all=TRUE)
    # sort increasing to allow correct derivation of dividend
    data.combine <- data.combine[order(data.combine$Date),]
    if (is.na(data.combine$EPS[1])) 
        data.combine$EPS[1] <- tail(data.eps.annual$EPS, n=1)
    data.combine$EPS <- na.locf(data.combine$EPS)
    data.combine <- na.omit(data.combine)    
}

In [ ]:
write.table(data.combine, file = 'Historical_PE_Ratio.csv', sep = ",", col.names = TRUE, row.names = FALSE, quote = FALSE)